### **HandEye Calibration**

##### **Table Of Contents**
- [Introduce](#Introduce)
- [Camera Calibration](#Camera-Calibration)
- [HandEye Calibration with Robot](#HandEye-Calibration-With-Robot)
    - [Camera Pose](#Camera-Pose)
    - [Robot Pose](#Robot-Pose)
    - [Compute Matrix Camera to Gripper](#Compute-Matrix-Camera-to-Gripper)
- [Results](#Results)

##### **Introduce**

Hand-eye calibration là quá trình xác định mối quan hệ hình học giữa hệ tọa độ của robot (robot base hoặc gripper) và hệ tọa độ của camera (camera gắn trên robot hoặc đặt cố định).  
Đây là bước quan trọng để đảm bảo rằng robot có thể chính xác thực hiện các thao tác dựa trên dữ liệu từ camera.

Có hai kiểu HandEye Calibration : 
- EyeToHand (camera gắn cố đinh, gripper + vật thể trong vùng quan sát của camera)
- EyeInHand (camera gắn trên gripper, di chuyển cùng gripper)

Dưới đây là các bước cụ thể để thực hiện hand-eye calibration:
- Thiết lập phần cứng : robot, camera, bảng hiệu chuẩn(checker board)
- Thực hiện camera calibration trước để có thông số camera_matrix và dist_coeffs
- Chụp dữ liệu:
    - Di chuyển gripper(kèm checker board) tới các vị trí khác nhau (5-9 hoặc nhiều hơn)
    - Tại mỗi vị trí thu thập lại pose của gripper + pose của camera
- Tính toán ma trận chuyển camera_to_gripper(camera_to_base) dựa trên dữ liệu poses thu thập


##### **Camera Calibration**

- Trước khi HandEye, cần hiệu chỉnh lại camera trước.  
- Tài liệu tham khảo : https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html  
- Các bước làm:
    - Chuẩn bị checker board (với kích thước ô vuông đã biết, vd: 25mm)
    - Chụp lại checker board (15 or more) ở các vị trí khác nhau, đặc biệt là vùng biên
    - Tính toán các thông số ma trận nội tại của camera(camera intrinsic) and hệ số méo (distortion coefficients)

Ma trận nội tại:

$$
K = 
\begin{bmatrix}
f_x & 0 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1
\end{bmatrix}
$$  

Hệ số méo:
$$
d = [k_1, k_2, p_1, p_2, k_3]
$$

Méo xuyên tâm(Radial distortion):   
$$
x' = x (1 + k_1 r^2 + k_2 r^4 + k_3 r^6)  
$$
$$
y' = y (1 + k_1 r^2 + k_2 r^4 + k_3 r^6)
$$

Méo tiếp tuyến(Tangential distortion):  
$$ 
x' = x + [2p_1 xy + p_2(r^2 + 2x^2)]
$$
$$
y' = y + [p_1(r^2 + 2y^2) + 2p_2 xy]
$$

Khoảng cách xuyên tâm \( r \) được tính như sau:

$$
r = \sqrt{x^2 + y^2}
$$

Tọa độ (x, y) trong hệ tọa độ ảnh chuẩn hóa được tính từ tọa độ pixel (u, v):

$$
x = \frac{u - c_x}{f_x}, \quad y = \frac{v - c_y}{f_y}
$$



Triển khai với code Opencv Python
```python

import numpy as np
import cv2 as cv
import glob

def calibrate_camera(image_dir, patter_size=(7,6), square_size=1):
    '''
    Calibrate camera using chessboard images
    :param image_dir: directory containing chessboard images
    :param patter_size: tuple of number of corners, ex: (7, 6) is 7 rows and 6 columns
    :param square_size: size of square in mm
    '''
    # termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((patter_size[1]*patter_size[0], 3), np.float32)
    objp[:,:2] = np.mgrid[0:patter_size[0], 0:patter_size[1]].T.reshape(-1,2)
    objp = objp * square_size

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    images = glob.glob(f'{image_dir}/*.jpg')

    for fname in images:
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(gray, patter_size, None)

        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)

            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners2)

            # Draw and display the corners
            cv.drawChessboardCorners(img, patter_size, corners2, ret)
            cv.imshow('img', img)
            cv.waitKey(500)
    cv.destroyAllWindows()

    # Calibrate camera
    # mtx: camera matrix, dist: distortion coefficients, rvecs: rotation vectors, tvecs: translation vectors
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return mtx, dist

```

##### **HandEye Calibration with Robot**

Sau khi đã có camera_matrix and dist_coeffs ở bước Calibration ở trên.  
Lúc này, ta sẽ tìm ma trận chuyển từ camera sang gripper(hoặc robot base).   
Cần thu thập 5->9 poses hoặc nhiều hơn của camera và gripper.  
- Các bước thực hiện(9 poses):  
    - Teaching 9 poses Robot  
    - Di duyển Robot(kèm theo checker board) tới từng pose, tại đó tính toán pose của camera,  
            thu thập lại robot pose và camera pose  
    - Sau khi thu thập đủ 9 pose -> tính toán ma trận chuyển từ camera sang gripper.


##### **Camera Pose**
Bao gồm ma trận xoay + vector tịnh tiến

Dữ liệu cần biết : 
- Ma trận nội tại K.
- Hệ số méo dist_coeffs.
- toạ độ 3D world_points ( trong thế giới thật) các điểm tham chiếu.
- toạ độ 2D image_points trong ảnh.

```python  

retval, rvec, tvec = cv2.solvePnP(world_points, image_points, K, dist_coeffs)
R, _ = cv2.Rodrigues(rvec)

```
R, tvec chính là thông tin pose của camera: 
- R là ma trận xoay
- tvec là ma trận tịnh tiến

##### **Robot Pose**

Lấy thông tin Robot Pose từ Bộ điều khiển Robot.

##### **Compute Matrix Camera to Gripper**

Sau khi thu thập đủ poses(9 Poses) của camera và Robot.
Ta tiến hành tính toán để tìm ra ma trận chuyển từ Camera to Gripper.

```python
R_cam_to_gripper, t_cam_to_gripper = cv2.calibrateHandEye(
            list_r_gripper_to_base, list_t_gripper_to_base,
            list_r_target_to_camera, list_t_target_to_camera
        )

T_cam_to_gripper = np.eye(4)
T_cam_to_gripper[:3, :3] = R_cam_to_gripper
T_cam_to_gripper[:3, 3] = t_cam_to_gripper.ravel()

return T_cam_to_gripper
```

##### **Results**

Sau khi có ma trận chuyển từ camera sang gripper (hoặc base).    
Ta sẽ tính toán như thế nào để chuyển từ toạ độ ảnh sang toạ độ Robot?  
Giả sử vật thể có tâm trong ảnh là (u, v) đơn vị pixel.

- **Bước 1**:  
Chuyển (u, v) sang toạ độ camera.  
$$
\begin{bmatrix}
X_{camera} \\
Y_{camera} \\
Z_{camera}
\end{bmatrix}
= z \cdot K^{-1} \cdot
\begin{bmatrix}
u \\
v \\
1
\end{bmatrix}
$$



*Note*:  
    - z : là depth của vật thể trong thực tế, nếu làm việc tại mặt phẳng Oxy cố định, ta coi z=1.  
    - K_inv là ma trận nghịch đảo của ma trận nội tại K của camera. 
    $$
    K^{-1} = \begin{bmatrix}
    \frac{1}{f_x} & 0 & -\frac{c_x}{f_x} \\
    0 & \frac{1}{f_y} & -\frac{c_y}{f_y} \\
    0 & 0 & 1
    \end{bmatrix}
    $$


- **Bước 2**:  
    - **Trường hợp EyeToHand**  
    Ma trận chuyển từ camera sang base, T_camera_to_base.
        $$
        \begin{bmatrix}
        X_{robot} \\
        Y_{robot} \\
        Z_{robot} \\
        1
        \end{bmatrix}
        = T_{camera\_to\_base} \cdot
        \begin{bmatrix}
        X_{camera} \\
        Y_{camera} \\
        Z_{camera} \\
        1
        \end{bmatrix}
        $$

    - **Trường hợp EyeInHand**  
        Ma trận chuyển từ camera sang gripper, T_camera_to_gripper.
        $$
        \begin{bmatrix}
        X_{gripper} \\
        Y_{gripper} \\
        Z_{gripper} \\
        1
        \end{bmatrix}
        = T_{camera\_to\_gripper} \cdot
        \begin{bmatrix}
        X_{camera} \\
        Y_{camera} \\
        Z_{camera} \\
        1
        \end{bmatrix}
        $$

        $$
        \begin{bmatrix}
        X_{robot} \\
        Y_{robot} \\
        Z_{robot} \\
        1
        \end{bmatrix}
        = T_{gripper\_to\_base} \cdot
        \begin{bmatrix}
        X_{gripper} \\
        Y_{gripper} \\
        Z_{gripper} \\
        1
        \end{bmatrix}
        $$

- **Bước 3**:  
Gửi toạ độ (X, Y, Z) cho Robot.


Thực hành với code OpenCV + Python

In [25]:
import numpy as np
import cv2

np.random.seed(42)

R = np.random.rand(3, 3)
t = np.random.randint(0, 100, size=(3, )).astype(np.float64)
T_camera_to_base = np.eye(4, 4)
T_camera_to_base[:3, :3] = R
T_camera_to_base[:3, 3] = t
print("T_camera_to_base: ")
print(T_camera_to_base)

fx, fy = (12., 12.)
cx, cy = (100., 100.)
camera_matrix = np.array(([fx, 0, cx], [0, fy, cy], [0, 0, 1]))
dist_coeffs = np.random.randn(5)

print("\n")
print("Camera matrix: ")
print(camera_matrix)
print("Dist coeffs: ")
print(dist_coeffs)
# image coordinates of P
print("\nImage coordinates: ")
P_image = (50, 200)
print(P_image)

P_image_ = np.append(P_image, 1)
# camera coordiantes
z_depth = 2
P_camera = z_depth * np.linalg.inv(camera_matrix) @ P_image_

P_camera_ = np.append(P_camera, 1)
P_robot = T_camera_to_base @ P_camera_

print("\nRobot coordinates: ")
print(P_robot)

T_camera_to_base: 
[[ 0.37454012  0.95071431  0.73199394 23.        ]
 [ 0.59865848  0.15601864  0.15599452  2.        ]
 [ 0.05808361  0.86617615  0.60111501 21.        ]
 [ 0.          0.          0.          1.        ]]


Camera matrix: 
[[ 12.   0. 100.]
 [  0.  12. 100.]
 [  0.   0.   1.]]
Dist coeffs: 
[ 0.07983181 -0.1595165   0.02222183 -0.42779291 -0.53181741]

Image coordinates: 
(50, 200)

Robot coordinates: 
[37.18805867 -0.07652099 36.15446902  1.        ]
